**Situación**
La congestión y el tráfico en las ciudades es un fenómeno natural que emerge de las distintas actividades urbanas y su concentración espacial.

**Complicación**
El transporte de personas también conlleva accidentes, con la correspondiente pérdida de vidas, problemas de salud y lesiones a las personas involucradas, daño a la propiedad pública y privada, y costo de tiempo a personas no involucradas.

**Propuesta**
Caracterizar la accidentabilidad en Santiago para apoyar el diseño de políticas públicas que reduzcan la cantidad de accidentes.

In [ ]:
import geopandas as gpd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from aves.data import census
from pathlib import Path
from aves.features.geo import clip_area_geodataframe
from aves.visualization.figures import small_multiples_from_geodataframe
from aves.visualization.maps import choropleth_map

sns.set_context('paper')
sns.set_style('ticks')

## Construcción del Dataset

In [ ]:
AVES_ROOT = Path("..") / ".." / ".."

https://mapas-conaset.opendata.arcgis.com/

https://mapas-conaset.opendata.arcgis.com/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/explore

In [ ]:
data_path = AVES_ROOT / "data" / "external" / "conaset"

if not data_path.exists():
    data_path.mkdir(parents=True)

In [ ]:
file_path = data_path / "2019-rm-accidents.json"

if not file_path.exists():
    import requests
    r = requests.get('https://opendata.arcgis.com/api/v3/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')
    with open(file_path, 'wt') as f:
        f.write(r.text)

In [ ]:
gdf = gpd.read_file(file_path, dtypes={'Fecha': 'str', 'Hora': 'str'}).to_crs('epsg:5361')

In [ ]:
gdf

In [ ]:
gdf.plot()

In [ ]:
gdf.columns

## Limpiado, Filtrado y Pre-Procesamiento

In [ ]:
comunas = census.read_census_map(
    "comuna", path=AVES_ROOT / "data" / "external" / "censo_2017_R13"
).to_crs("epsg:5361")

zones = (
    gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
    .set_index("ID")
    .to_crs(comunas.crs)
)

comunas_urbanas = (
    comunas[comunas["COMUNA"].isin(zones["Com"].unique())]
    .drop("NOM_COMUNA", axis=1)
    .copy()
)

comunas_urbanas["NombreComuna"] = comunas_urbanas["COMUNA"].map(
    dict(zip(zones["Com"], zones["Comuna"].str.upper()))
)

bounding_box = zones.total_bounds
comunas_urbanas = clip_area_geodataframe(
    comunas_urbanas, zones.total_bounds, buffer=1000
)

comunas_urbanas.plot()

In [ ]:
comunas_urbanas["NombreComuna"] = comunas_urbanas["NombreComuna"].replace(
    {"Á": "A", "Ú": "U", "Ó": "O", "Í": "I", "É": "E"}, regex=True
)


In [ ]:
gdf[['COMUNAREAL', 'Comuna_1']].value_counts()

In [ ]:
len(gdf[gdf['COMUNAREAL'] != gdf['Comuna_1']]) / len(gdf)

In [ ]:
ax = comunas_urbanas.plot(edgecolor="black", facecolor="none")
gdf[(gdf["COMUNAREAL"] != gdf["Comuna_1"]) & (gdf["Comuna_1"] == "SANTIAGO")].plot(
    column="COMUNAREAL", legend=False, ax=ax
)


In [ ]:
ax = comunas_urbanas.plot(edgecolor="black", facecolor="none")
gdf[(gdf["COMUNAREAL"] != gdf["Comuna_1"]) & (gdf["Comuna_1"] == "SANTIAGO")].plot(
    column="Comuna_1", legend=False, ax=ax
)

In [ ]:
scl_accidents = gpd.sjoin(gdf.to_crs(comunas_urbanas.crs), comunas_urbanas, op='within')

scl_accidents.plot(alpha=0.01)

In [ ]:
scl_accidents['Fallecidos'].value_counts()

In [ ]:
scl_accidents['Graves'].value_counts()

In [ ]:
scl_accidents['Menos_Grav'].value_counts()

In [ ]:
scl_accidents['Ilesos'].value_counts()

In [ ]:
scl_accidents['Leves'].value_counts()

In [ ]:
scl_accidents['victimas'] = gdf[['Ilesos', 'Graves', 'Menos_Grav', 'Fallecidos', 'Leves']].sum(axis=1)

In [ ]:
scl_accidents['victimas'].value_counts(sort=False).sort_index().plot(kind='bar', logy=True)

In [ ]:
scl_accidents['Fecha']

In [ ]:
scl_accidents['fecha'] = pd.to_datetime(gdf['Fecha'])

In [ ]:
scl_accidents['Hora']

## ¿Qué contiene?

In [ ]:
gdf.columns

In [ ]:
scl_accidents['Condición'].value_counts()

In [ ]:
scl_accidents['Tipo__CONA'].value_counts()

In [ ]:
scl_accidents['Causa__CON'].value_counts()

## ¿Cuándo?

In [ ]:
scl_accidents.resample('1d', on='fecha').size().plot()

In [ ]:
scl_accidents.resample('1w', on='fecha').size().plot()

In [ ]:
scl_accidents['fecha']

## ¿Dónde?

In [ ]:
from aves.visualization.maps import heat_map, heatmap
from aves.visualization.figures import small_multiples_from_geodataframe

fig, ax = small_multiples_from_geodataframe(comunas_urbanas, 1, height=7)

comunas_urbanas.plot(ax=ax, facecolor="#efefef", edgecolor="none")
heat_map(ax, scl_accidents, weight="victimas", low_threshold=0.05)
comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="white")


In [ ]:
from aves.features.utils import normalize_rows
victimas_por_comuna = scl_accidents.groupby(['COMUNAREAL', 'Tipo__CONA'])['victimas'].sum().unstack(fill_value=0).pipe(normalize_rows)
sns.clustermap(victimas_por_comuna, method='ward')

In [ ]:
scl_accidents['Ubicación'].value_counts()

## ¿Cómo?

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Ubicación")["Tipo__CONA"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows)
)

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Tipo__CONA")["victimas"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows)
)


In [ ]:
sns.clustermap(
    scl_accidents.groupby("Tipo__CONA")["Causa__CON"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows)
)

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Ubicación")["Causa__CON"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows)
)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

mean_accidents = (
    lambda name, x: x.resample("1d", on="fecha")
    # .size()
    ["victimas"]
    .sum()
    .rolling(14, center=True)
    .mean()
    .rename(name)
    .to_frame()
    .apply(lambda x: (x - x.mean()) / x.std())
)

for ax, cat in zip(axes, ["COLISION", "CHOQUE", "ATROPELLO"]):
    # print(g)
    g = scl_accidents[scl_accidents["Tipo__CONA"] == cat]
    mean_accidents("all", scl_accidents).plot(ax=ax, color="grey", linewidth=0.5, legend=False)
    mean_accidents(cat, g).plot(ax=ax, linewidth=1.5, color="purple", legend=False)
    ax.set_title(cat)
    sns.despine(ax=ax, left=True, bottom=True)

fig.tight_layout()


**¿Qué concluyen?**